this notebook is used for scarping the Wiki page about Toronto neighbourhoods.

In [1]:
#import package
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Then we create a soup object to collect the content of wiki page. 

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup= BeautifulSoup(page.content,'html.parser')

By inspecting the html code of this wikipidia page, we located the table as "wikitable sortable"

In [3]:
#find the table and extract the rows
wikidf = soup.find('table',{'class':"wikitable sortable"})
rows=wikidf.findAll('tr')
rows

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>,
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>,
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>,
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>,
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>,
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>,
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>,
 <tr>
 <td>M2B
 </td>
 <td>Not assigned
 </td>

Now, we have all the content of table, and then we are going to split them and store in sepereate list.

In [4]:
postal=[]
borough=[]
neighbourhood=[]
for row in rows:
    cells=row.findAll('td')
    if cells!=[]:
    
        postal.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighbourhood.append(cells[2].find(text=True))

In [5]:
postal[0:5]

['M1A\n', 'M2A\n', 'M3A\n', 'M4A\n', 'M5A\n']

In [6]:
borough[0:5]

['Not assigned\n',
 'Not assigned\n',
 'North York\n',
 'North York\n',
 'Downtown Toronto\n']

In [7]:
neighbourhood[0:5]

['Not assigned\n',
 'Not assigned\n',
 'Parkwoods\n',
 'Victoria Village\n',
 'Regent Park, Harbourfront\n']

We realized that the string we collected contains '\n', and we should remove them.

In [8]:
postal=[str(x).replace('\n','') for x in postal]
borough=[str(x).replace('\n','') for x in borough]
neighbourhood=[str(x).replace('\n','') for x in neighbourhood]

In [9]:
postal[0:5]

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']

In [10]:
borough[0:5]

['Not assigned',
 'Not assigned',
 'North York',
 'North York',
 'Downtown Toronto']

In [11]:
neighbourhood[0:5]

['Not assigned',
 'Not assigned',
 'Parkwoods',
 'Victoria Village',
 'Regent Park, Harbourfront']

Now we are ready to build the pandas dataframe:

In [12]:
colname=['PostalCode','Borough','Neighborhood']
df=pd.DataFrame(columns=colname)
df['PostalCode']=postal
df['Borough']=borough
df['Neighborhood']=neighbourhood
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [13]:
df.shape

(180, 3)

Remove the rows with no 'Borough'.

In [14]:
df=df[df['Borough']!='Not assigned']
df.shape

(103, 3)

If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [15]:
df[df['Neighborhood']=='Not assigned']['Neighborhood']=df[df['Neighborhood']=='Not assigned']['Borough']
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [16]:
df.shape

(103, 3)

In [22]:
! pip install geocoder
import geocoder # import geocoder

read the existing dataset about the geographical location of each community in toronto.

In [39]:
location=pd.read_csv('Geospatial_Coordinates.csv')
location

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merge this table with the table we created before, join on the postal code column.

In [42]:
df_merge=df.join(location.set_index('Postal Code'),on='PostalCode')
df_merge

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
